## Currency conversion tool

In [ ]:
# Importing all the required modules and classes

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import requests

c:\Users\Gaurav\OneDrive\Desktop\GenAI_Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# creating the tools

from langchain_core.tools import InjectedToolArg
from typing import Annotated

""" InjectedToolArg assures that input argument of the second function where it is applied comes from output of the first function. 
    In this case argument 'conversion_rate' of tool function 'convert' is assigned by outcome of the first function"""

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:

    """This function fetches the currency conversion factor between a given base currency and a target currency"""
    
    url = f"https://v6.exchangerate-api.com/v6/e3fc4b82e5f0bd3700065d92/pair/{base_currency}/{target_currency}"

    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:

    """Given a currency conversion rate, this function calculates the target currency value from a given base currency value"""

    return base_currency_value * conversion_rate

In [ ]:
# instantiating the llm (openai in this case) and binding the llm with tools just created above

llm = ChatOpenAI()

llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [ ]:
# Query from the user is tagger as 'HumanMessage' as we want to keep track of different responses seperately 
# i.e. 'Human Message', 'AI Message' & 'Tool Message'

query_1 = HumanMessage("can you get the conversion factor between USD and INR, and also convert 10.5 USD into INR")
messages = [query_1]
messages

[HumanMessage(content='can you get the conversion factor between USD and INR, and also convert 10.5 USD into INR', additional_kwargs={}, response_metadata={})]

In [ ]:
# Invoking the llm to get AIMessage 

ai_message = llm_with_tools.invoke(messages)
ai_message

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 122, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CwOCILrg9cybwYKsaagjHr9ATCzlu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019ba6ec-7657-7042-99c0-55ab564cce04-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_hUFQboAw8OS0OlFbyBJmRKkn', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10.5}, 'id': 'call_rZ0tLV9ndWiySh9VBHq4pCYr', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 122, 'output_tokens

In [ ]:
# AIMessage appended to list 'messages' to keep complete chat context or history for final response
messages.append(ai_message)

In [7]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_hUFQboAw8OS0OlFbyBJmRKkn',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10.5},
  'id': 'call_rZ0tLV9ndWiySh9VBHq4pCYr',
  'type': 'tool_call'}]

In [ ]:
# Running a loop over different tool calls in ai message (response from invoking llm). Since 2 tools were created and called
# we get 2 dictionary objects wrapped in a string from where 2nd argument i.e. "conversion_rate" of tool 'convert' is extracted

import json

for tool_call in ai_message.tool_calls:
    # execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)

        # fetch the conversion rate
        conversion_rate = json.loads(tool_message1.content)["conversion_rate"] # "tool_message1.content" is a 'str' NOT a dictionary so can't 
                                                                               # fetch conversion_rate directly using tool_message.content["conversion_rate"]
                                                                               # that's why it is first converted to dictionary data type and then 
                                                                               # conversion_rate is accessed from this dictionary
        # append this tool message to messages_list
        messages.append(tool_message1)
        
    # execute the 2nd tool using the conversion rate from tool1
    if tool_call['name'] == 'convert':
        tool_call['args']['conversion_rate'] = conversion_rate # assigning the conversion_rate obtained from tool1 to 'args' dictionary in tool_call2
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [ ]:
# 'messages' list now containing complete context i.e. 'Human message', 'ai message' and 'tool message'
messages

[HumanMessage(content='can you get the conversion factor between USD and INR, and also convert 10.5 USD into INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 122, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CwOCILrg9cybwYKsaagjHr9ATCzlu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019ba6ec-7657-7042-99c0-55ab564cce04-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_hUFQboAw8OS0OlFbyBJmRKkn', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base

In [ ]:
# invoking the llm equipped with tools to invoke the final response by using messages list 
# to provide complete context or history to the LLM

llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is approximately 90.2185.\n\nWhen converting 10.5 USD into INR, the value is approximately 947.29425 INR.'